In [ ]:
!pip install opendatasets
!pip install pandas
!pip install kaggle

In [ ]:
import opendatasets as od
import pandas

od.download("https://www.kaggle.com/datasets/jillanisofttech/fake-or-real-news")

Skipping, found downloaded files in "./fake-or-real-news" (use force=True to force download)


In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import numpy as np
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.optimizers import Adam


def clean_text(text):
    text = re.sub('\[[^]]*\]', '', text)  # remove text in brackets
    text = re.sub(r'http\S+', '', text)  # remove URLs
    stop_words = set(stopwords.words('english'))  # Use NLTK's stopwords
    punctuation = list(string.punctuation)
    stop_words.update(punctuation)  # Add punctuation to stopwords list
    text = " ".join(word for word in text.split() if word.lower() not in stop_words)
    return text


data_path = '/content/fake-or-real-news/fake_or_real_news.csv'  # Update this path with your CSV file
data = pd.read_csv(data_path)


data['text'] = data['text'].apply(clean_text)


tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data['text'])
sequences = tokenizer.texts_to_sequences(data['text'])
X = pad_sequences(sequences, maxlen=500)


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['label'])
y = y.reshape(-1, 1)


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


vocab_size = 5000
embedding_dim = 400
hidden_dim = 256
output_dim = 1

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=500))
model.add(LSTM(hidden_dim, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(hidden_dim, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(output_dim, activation='sigmoid'))





In [ ]:
from keras.callbacks import ModelCheckpoint
model_checkpoint_path = '/content/drive/MyDrive/lab1_mlops_weights/model.h5'
checkpoint = ModelCheckpoint(model_checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_data=(X_val, y_val), callbacks=[checkpoint])

Epoch 1/3
159/159 [==============================] - ETA: 0s - loss: 0.3916 - accuracy: 0.8275
Epoch 1: val_loss improved from inf to 0.54284, saving model to /content/drive/MyDrive/lab1_mlops_weights/model.h5
159/159 [==============================] - 578s 4s/step - loss: 0.3916 - accuracy: 0.8275 - val_loss: 0.5428 - val_accuracy: 0.7348
Epoch 2/3


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


159/159 [==============================] - ETA: 0s - loss: 0.2597 - accuracy: 0.8970
Epoch 2: val_loss improved from 0.54284 to 0.29211, saving model to /content/drive/MyDrive/lab1_mlops_weights/model.h5
159/159 [==============================] - 567s 4s/step - loss: 0.2597 - accuracy: 0.8970 - val_loss: 0.2921 - val_accuracy: 0.8856
Epoch 3/3
159/159 [==============================] - ETA: 0s - loss: 0.1136 - accuracy: 0.9597
Epoch 3: val_loss did not improve from 0.29211
159/159 [==============================] - 567s 4s/step - loss: 0.1136 - accuracy: 0.9597 - val_loss: 0.3141 - val_accuracy: 0.8958


In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)


40/40 [==============================] - 12s 288ms/step - loss: 0.3141 - accuracy: 0.8958
Validation Loss: 0.314130961894989
Validation Accuracy: 0.8958168625831604
